<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

### **Opciones para fuentes externas**
Mientras que la consulta directa de archivos funciona bien para formatos autodescriptivos, muchas fuentes de datos requieren configuraciones adicionales o la declaración de schema para ingerir correctamente los registros.

En esta lección, crearemos tablas utilizando fuentes de datos externas. Si bien estas tablas aún no se almacenarán en el formato de Delta Lake (y, por lo tanto, no estarán optimizadas para Lakehouse), esta técnica ayuda a facilitar la extracción de datos de diversos sistemas externos.

#### **Objetivos de aprendizaje**
Al final de esta lección, deberías ser capaz de:
- Utilizar Spark SQL para configurar las opciones de extracción de datos de fuentes externas
- Crear tablas contra fuentes de datos externas para diversos formatos de archivo
- Describir el comportamiento predeterminado al consultar tablas definidas contra fuentes externas

#### **Run Setup**

El setup script creará los datos y declarará los valores necesarios para que el resto de este notebook se ejecute.

In [ ]:
%run ../Includes/Classroom-Setup-02.2

#### **Cuando las consultas directas no funcionan**

Aunque las vistas pueden utilizarse para realizar consultas directas a archivos entre sesiones, su utilidad es limitada.

Los archivos CSV son uno de los formatos de archivo más comunes, pero una consulta directa a estos archivos rara vez devuelve los resultados deseados.

In [ ]:
%sql
SELECT * FROM csv.`${DA.paths.sales_csv}`

<center><img src="https://i.postimg.cc/4xSLxK5r/db347.png"></center>

De lo anterior se desprende que:
1. La fila de cabecera se está extrayendo como una fila de tabla
1. Todas las columnas se cargan como una sola columna
1. El archivo está delimitado por pipes (**`|`**)
1. La última columna parece contener datos anidados que se truncan.

#### **Registro de tablas en datos externos con opciones de lectura**

Mientras que Spark extraerá algunas fuentes de datos autodescriptivas de forma eficiente utilizando la configuración por defecto, muchos formatos requerirán la declaración del schema u otras opciones.

Aunque hay muchas <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-create-table-using.html" target="_blank">configuraciones adicionales</a> que puede establecer al crear tablas contra fuentes externas, la sintaxis que se muestra a continuación demuestra lo esencial necesario para extraer datos de la mayoría de los formatos.

<strong><code>
CREATE TABLE table_identifier (col_name1 col_type1, ...)<br/>
USING data_source<br/>
OPTIONS (key1 = val1, key2 = val2, ...)<br/>
LOCATION = path<br/>
</code></strong>

Tenga en cuenta que las opciones se pasan con las keys como texto sin entrecomillar y los valores entre comillas. Spark soporta muchas <a href="https://docs.databricks.com/data/data-sources/index.html" target="_blank">fuentes de datos</a> con opciones personalizadas, y sistemas adicionales pueden tener soporte no oficial a través de <a href="https://docs.databricks.com/libraries/index.html" target="_blank">librerías externas</a>. 

**NOTA**: Dependiendo de la configuración de su workspace, es posible que necesite ayuda del administrador para cargar librerías y configurar los parámetros de seguridad necesarios para algunas fuentes de datos.

La celda a continuación demuestra el uso de Spark SQL DDL para crear una tabla contra una fuente CSV externa, especificando:
1. Los nombres y tipos de columnas
1. El formato del archivo
1. El delimitador utilizado para separar los campos
1. La presencia de una cabecera
1. La ruta donde se almacenan los datos

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS sales_csv (
  order_id LONG, 
  email STRING, 
  transactions_timestamp LONG, 
  total_item_quantity INTEGER, 
  purchase_revenue_in_usd DOUBLE, 
  unique_items INTEGER, 
  items STRING)
USING CSV
OPTIONS (
  header = "true",
  delimiter = "|"
)
LOCATION "${DA.paths.sales_csv}"

**NOTA:** Para crear una tabla contra una fuente externa en PySpark, puedes envolver este código SQL con la función **`spark.sql()`**.

In [ ]:
%python
spark.sql(f"""
CREATE TABLE IF NOT EXISTS sales_csv (
  order_id LONG, 
  email STRING, 
  transactions_timestamp LONG, 
  total_item_quantity INTEGER, 
  purchase_revenue_in_usd DOUBLE, 
  unique_items INTEGER, 
  items STRING)
USING CSV
OPTIONS (
  header = "true",
  delimiter = "|"
)
LOCATION "{DA.paths.sales_csv}"
""")

Observe que no se ha movido ningún dato durante la declaración de la tabla. 

Al igual que cuando consultamos directamente nuestros archivos y creamos una vista, seguimos apuntando a archivos almacenados en una ubicación externa.

Ejecute la siguiente celda para confirmar que los datos se están cargando correctamente.

In [ ]:
%sql
SELECT * FROM sales_csv

<center><img src="https://i.postimg.cc/JnWQsjBd/db348.png"></center>

In [ ]:
%sql
SELECT COUNT(*) FROM sales_csv

<center><img src="https://i.postimg.cc/Rh4GDwSn/db349.png"></center>

Todos los metadatos y opciones pasados durante la declaración de la tabla se persistirán en el metastore, asegurando que los datos en la ubicación siempre se leerán con estas opciones.

**NOTA**: Cuando se trabaja con CSV como fuente de datos, es importante asegurarse de que el orden de las columnas no cambia si se añaden archivos de datos adicionales al directorio de origen. Debido a que el formato de datos no tiene una fuerte aplicación del schema, Spark cargará las columnas y aplicará los nombres de las columnas y los tipos de datos en el orden especificado durante la declaración de la tabla.

Al ejecutar **`DESCRIBE EXTENDED`** en una tabla se mostrarán todos los metadatos asociados a la definición de la tabla.

In [ ]:
%sql
DESCRIBE EXTENDED sales_csv

<center><img src="https://i.postimg.cc/j2M6393G/db350.png"></center>

#### **Límites de Tablas con Fuentes de Datos Externas**

Si usted ha tomado otros cursos sobre Databricks o revisado cualquier literatura de nuestra compañía, usted puede haber oído hablar de Delta Lake y Lakehouse. Tenga en cuenta que siempre que estemos definiendo tablas o consultas contra fuentes de datos externas, **no podemos** esperar las garantías de rendimiento asociadas a Delta Lake y Lakehouse.

Por ejemplo: mientras que las tablas de Delta Lake garantizarán que siempre se consulte la versión más reciente de los datos de origen, las tablas registradas contra otras fuentes de datos pueden representar versiones anteriores almacenadas en caché.

La siguiente celda ejecuta cierta lógica que podemos considerar como la representación de un sistema externo que actualiza directamente los archivos subyacentes a nuestra tabla.

In [ ]:
%python
(spark.read
      .option("header", "true")
      .option("delimiter", "|")
      .csv(DA.paths.sales_csv)
      .write.mode("append")
      .format("csv")
      .save(DA.paths.sales_csv))

Si miramos el recuento actual de registros en nuestra tabla, el número que veremos no reflejará estas filas recién insertadas.

In [ ]:
%sql
SELECT COUNT(*) FROM sales_csv

<center><img src="https://i.postimg.cc/0Qd0DrY5/db351.png"></center>

En el momento en que consultamos previamente esta fuente de datos, Spark almacenó automáticamente en caché los datos subyacentes en el almacenamiento local. Esto asegura que en consultas posteriores, Spark proporcionará el rendimiento óptimo simplemente consultando esta caché local.

Nuestra fuente de datos externa no está configurada para indicar a Spark que debe actualizar estos datos. 

**Podemos** refrescar manualmente la caché de nuestros datos ejecutando el comando **`REFRESH TABLE`**.

In [ ]:
%sql
REFRESH TABLE sales_csv

Tenga en cuenta que la actualización de la tabla invalidará la caché, lo que significa que tendremos que volver a escanear la fuente de datos original y recuperar todos los datos de la memoria. 

En el caso de conjuntos de datos muy grandes, esto puede llevar bastante tiempo.

In [ ]:
%sql
SELECT COUNT(*) FROM sales_csv

<center><img src="https://i.postimg.cc/W4Znp5tk/db352.png"></center>

#### **Extracción de datos de bases de datos SQL**
Las bases de datos SQL son una fuente de datos extremadamente común, y Databricks tiene un driver JDBC estándar para conectarse con muchos tipos de SQL.

La sintaxis general para crear estas conexiones es:

<strong><code>
CREATE TABLE <jdbcTable><br/>
USING JDBC<br/>
OPTIONS (<br/>
&nbsp; &nbsp; url = "jdbc:{databaseServerType}://{jdbcHostname}:{jdbcPort}",<br/>
&nbsp; &nbsp; dbtable = "{jdbcDatabase}.table",<br/>
&nbsp; &nbsp; user = "{jdbcUsername}",<br/>
&nbsp; &nbsp; password = "{jdbcPassword}"<br/>
)
</code></strong>

En el siguiente ejemplo de código, nos conectaremos con <a href="https://www.sqlite.org/index.html" target="_blank">SQLite</a>.
  
**NOTA:** SQLite utiliza un archivo local para almacenar una base de datos, y no requiere un puerto, nombre de usuario o contraseña.
  
<img src="https://files.training.databricks.com/images/icon_warn_24.png"> **AVISO**: La configuración backend del servidor JDBC asume que está ejecutando este notebook en un single-node cluster. Si está ejecutando en un clúster con múltiples workers, el cliente que se ejecuta en los executors no será capaz de conectarse al driver.

In [ ]:
%sql
DROP TABLE IF EXISTS users_jdbc;

CREATE TABLE users_jdbc
USING JDBC
OPTIONS (
  url = "jdbc:sqlite:${DA.paths.ecommerce_db}",
  dbtable = "users"
)

Ahora podemos consultar esta tabla como si estuviera definida localmente.

In [ ]:
%sql
SELECT * FROM users_jdbc

<center><img src="https://i.postimg.cc/zfTCY3Pw/db353.png"></center>

Si observamos los metadatos de la tabla, veremos que hemos capturado la información del schema del sistema externo.

Las propiedades de almacenamiento (que incluirían el nombre de usuario y la contraseña asociados a la conexión) se redactan automáticamente.

In [ ]:
%sql
DESCRIBE EXTENDED users_jdbc

<center><img src="https://i.postimg.cc/G3Kjp83q/db354.png"></center>

Mientras que la tabla aparece como **`MANAGED`**, el listado del contenido de la ubicación especificada confirma que no se está persistiendo ningún dato localmente.

In [ ]:
%python
import pyspark.sql.functions as F

location = spark.sql("DESCRIBE EXTENDED users_jdbc").filter(F.col("col_name") == "Location").first()["data_type"]
print(location)

files = dbutils.fs.ls(location)
print(f"Found {len(files)} files")

<center><img src="https://i.postimg.cc/L8ctWjv6/db355.png"></center>

Tenga en cuenta que algunos sistemas SQL, como los Data Warehouse, tendrán drivers personalizados. Spark interactuará con varias bases de datos externas de manera diferente, pero los dos enfoques básicos se pueden resumir en:
1. Trasladar toda la tabla o tablas de origen a Databricks y luego ejecutar la lógica en el clúster actualmente activo.
1. Empujar la consulta a la base de datos SQL externa y sólo transferir los resultados de vuelta a Databricks

En cualquiera de los casos, trabajar con conjuntos de datos muy grandes en bases de datos SQL externas puede incurrir en una sobrecarga significativa debido a:
1. Latencia de transferencia de red asociada con el movimiento de todos los datos a través de la Internet pública
1. Ejecución de la lógica de consulta en sistemas de origen no optimizados para consultas de big data

Ejecute la siguiente celda para eliminar las tablas y archivos asociados a esta lección.

In [ ]:
%python
DA.cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>